## Машинный перевод на трансформере с использованием torchtext

Один из самых известных параллельных датасетов для машинного перевода - [Tatoeba](https://tatoeba.org/en/). В пригодном для обучения виде его можно найти [тут](https://www.manythings.org/anki/). Я для удобства сразу разделила файл на трейн и тест (можно это самостоятельно сделать в колабе):

In [ ]:
!wget -qq http://www.manythings.org/anki/rus-eng.zip 
!unzip rus-eng.zip

with open('rus.txt') as f:
    lines = f.readlines() 

eighty = int(len(lines) * 0.8)

with open('train.txt', 'w') as f:
    for line in lines[:eighty]:
        f.write(line)

with open('test.txt', 'w') as f:
    for line in lines[eighty:]:
        f.write(line)

Для того, чтобы удобным образом делать датасеты и даталоудеры в торче из текстов, существует библиотека torchtext. В ее ранних версиях (примерно до 0.6) вы можете видеть такие классы, как Field, Example, Dataset, BucketIterator - они все были выпилены из более поздних версий, так что если видите какой-нибудь туториал с этими классами, устанавливайте torchtext==0.6.0. 

В современных версиях разрабы торча, как обычно, полностью поломали всю обратную совместимость, какая и была, теперь у нас совсем поменялись принципы работы с текстовыми данными. 

Для того, чтобы обучить сеточку переводить, нам нужно будет сделать такой датасет, который будет плеваться парами предложений source-target, при этом они должны быть токенизированы и добиты паддингом. И не забудем, что нам нужен словарь: для того, чтобы эти самые токены превратить в индексы. 

Для создания словаря и для загрузки такого датасета с парами предложений у torchtext есть свои инструменты. Заимпортим их. Нам также понадобится spacy для токенизации, для него еще нужно загрузить соответствующие модельки:

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm

In [1]:
from torchtext.data.utils import get_tokenizer          # будет брать токенизатор из спейси и пихать его в collate_fn
import torchdata.datapipes as dp                        # будет делать итератор из нашего текстового файла
from torchtext.vocab import build_vocab_from_iterator   # будет делать словарь
from torch.nn.utils.rnn import pad_sequence             # будет падить
from torch.utils.data import DataLoader                 # будет грузить батчи
import spacy 

# заодно импортнем все нужное для модельки
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

# это просто время засекать
from timeit import default_timer as timer

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Потом нам надо будет задать некоторые важные вещи, которые мы будем использовать, и подготовить себе инструменты для токенизации и собственно словари (их по два - ведь у нас два языка. )

In [ ]:
# Зададим языки
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'ru'

token_transform = {}
vocab_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='ru_core_news_sm')

# Зададим спецсимволы, чтобы потом их раскодировывать
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Не перепутайте порядок - они в таком порядке и добавятся в словарь
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

Теперь подготовим итератор для датасета и соберем словарь. 

In [3]:
def getTokens(data_iter, place):
    """
    Возвращаем списки токенов для соответствующих языков: 
    наш итератор содержит пары язык-предложение, поэтому
    place=0 - это язык-источник, 
    place=1 - это язык-таргет.
    """
    for english, russian in data_iter:
        if place == 0:
            yield token_transform['en'](english)
        else:
            yield token_transform['ru'](russian)

def create_iter(path):
    """
    Создаем итератор по пути файла
    """
    pipe = dp.iter.IterableWrapper([path]) # эта штука нужна, чтобы не загружать содержимое файла целиком
    pipe = dp.iter.FileOpener(pipe, mode='rb') # приготавливаемся к чтению
    pipe = pipe.parse_csv(skip_lines=0, delimiter='\t', as_tuple=True) # парсим файл как тсв (у нас там действительно тсв-формат)
    pipe = pipe.map(lambda row: row[:2]) # оставляем только первые две ячейки, а то там дальше лицензия еще
    return pipe

# Создаем итераторы трейна и валидации
train = create_iter('train.txt')
test = create_iter('test.txt')

Проверим, что все ок:

In [ ]:
next(iter(train))

Собственно словарь собирается с помощью функции build_vocab_from_iterator:

In [5]:
vocab_transform['en'] = build_vocab_from_iterator(
    getTokens(train, 0), # вызываем нашу функцию токенизации
    min_freq=2, # отсеиваем слишком редкие слова
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'], # учитываем, что у нас 4 спецсимвола
    special_first=True) # и что они идут в начале словаря
vocab_transform['ru'] = build_vocab_from_iterator(
    getTokens(train, 1), 
    min_freq=2, 
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'], 
    special_first=True)
# дальше нужно еще установить дефолтный индекс для OOV-слов
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

Также нам понадобится маскировать наши предложения: трансформер не должен видеть правый контекст, он же генерирует новый текст, к тому же, маскировать нужно и паддинги, чтобы они не учитывались при оценке:

<img src="https://cdn.hashnode.com/res/hashnode/image/upload/v1638824585791/vkXCmdGyw.png?auto=compress,format&format=webp" width=500>

In [6]:
def generate_square_subsequent_mask(sz):
    """
    Генерирует маску: она выглядит как матричка-треугольник, где один угол заполнен бесконечностями. 
    """
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    """ 
    Эта функция использует ту, что мы задефайнили выше
    """
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    # заполняем верхний уголок
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len) 
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    # учитываем паддинги
    src_padding_mask = (src == PAD_IDX).transpose(0, 1) 
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Не забудем, что нам необходимо еще позаботиться о функции collate_fn. 

In [7]:
# просто вспомогательная функция-декоратор, чтобы применять другие функции сразу на много элементов
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# добавляет BOS/EOS и создает вектор с индексами инпутов
def tensor_transform(token_ids):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# трансформации для ``src`` и ``tgt``, которые собственно превращают наши сырые строки в тензора с индексами
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #токенизация
                                               vocab_transform[ln], #индексация токенов
                                               tensor_transform) # добавили BOS/EOS и создали тензор


# наконец collate_fn
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch: # scr_sample & tgt_sample - это две строки
        # отстрипим \n и прогоним через выше написанные функции
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    # отпадим
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

Наконец-то архитектура! 

Код написан разработчиками торча по статье [Attention is All You Need](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf), обязательно причаститесь. 

В жизни самостоятельно такое писать, скорее всего, не понадобится, все эти вещи уже реализованы за нас в библиотеках типа transformers. 

Саму имплементацию статьи с комментариями можно посмотреть [тут](https://nlp.seas.harvard.edu/2018/04/03/attention.html).

In [8]:
# вспомогательный класс для учета позиции слова в предложении
# выглядит жутко, но на самом деле это просто тригонометрическая формула, дропаут и residual connection
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        # формула
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        # сохраняем буфер для residual
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        # dropout + residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# конвертируем тензор индексов в соответствующий тензор с эмбеддингами токенов
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq с трансформером
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        # у нас есть слой - трансформер с энкодерами и декодерами
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        # просто голова для генерации токенов
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        # слои эмбеддингов
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        # позиции токенов
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        # делаем эмбеддинги
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        # все пихаем в трансформер, он сам разберется
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        # этот метод нужен для того, чтобы при использовании обученной модели брать выхлоп энкодера
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        # а этот достает декодер. Ему еще нужна память о предыдущих сгенерированных токенах
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

Зададим параметры для обучения.

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128 # на слишком большом размере умрет видюшка
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

# задаем инициализацию (будем использовать Ксавье-Глоро)
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

Подготовим петлю обучения. 

In [10]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = train
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        # у сорса отрезаем последний токен - он нам не нужен
        tgt_input = tgt[:-1, :]

        # делаем масочки
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        # получаем логиты - сырые недовероятности токенов
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        # у таргета отрезаем первый токен - тоже не нужен
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = test
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

Подготовим еще пару функций для отображения наших переводов:

In [11]:
# Обычный гриди декод (всегда будет выдавать один и тот же результат, нам сойдет, мы не чатбота пишем)
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# собственно будем вызывать ее для перевода
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

Ну и запустим обучение. У нас довольно большой датасет, и даже на 4 эпохах уже получится кое-какое качество. Можно в качестве ручной оценки еще и впихнуть в петлю обучения тестовое предложение для перевода:

In [ ]:
NUM_EPOCHS = 5 
SENT = 'I can translate! No, I can\'t'

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print(f"Переводим: {SENT}\t Результат: {translate(transformer, SENT)}")
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))